In [ ]:
import f3dasm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('science')

In [ ]:
dim = 3
bounds = np.tile([-1.0,1.0], (dim, 1))
design = f3dasm.make_nd_continuous_design(bounds=bounds, dimensionality=dim)
data = f3dasm.Data(design=design)

In [ ]:
args = {
'realizations': 3,
'iterations': 100,
'function': f3dasm.functions.Levy(dimensionality=dim, noise=None, scale_bounds=bounds),
'sampler': f3dasm.sampling.LatinHypercubeSampling(design=design),
'verbal': True
}



In [ ]:
args['function'].offset

In [ ]:
args['function'].plot(domain=bounds,px=100)

In [ ]:
# hyperparameters=[{'learning_rate': x} for x in np.logspace(-8,2,11)]
# hyperparameters


In [ ]:
# results = []

# for h in hyperparameters:
#     args_temp = args.copy()
#     args_temp['optimizer'] = f3dasm.optimization.Adamax(data=data, hyperparameters=h)
#     results.append(f3dasm.run_multiple_realizations(**args_temp))

In [ ]:
results = []

for opt in [f3dasm.optimization.Adam, f3dasm.optimization.Adam2]:
    args_temp = args.copy()
    args_temp['optimizer'] = opt(data=data)
    results.append(f3dasm.run_multiple_realizations(**args_temp))

In [ ]:
def calc_mean_std(results):
    mean_y = pd.concat([d.get_output_data().cummin() for d in results], axis=1).mean(axis=1)
    std_y = pd.concat([d.get_output_data().cummin() for d in results], axis=1).std(axis=1)
    return mean_y, std_y

In [ ]:
def plot_results(results, ax):
    mean_y, std_y = calc_mean_std(results.data)

    ax.plot(mean_y, label=f"optimizer={results.optimizer}")

    ax.fill_between(np.arange(len(mean_y)), mean_y-1.96*std_y, mean_y+1.96*std_y, alpha=.1)

    return ax

In [ ]:
fig, ax = plt.figure(figsize=(15,8)), plt.axes()

ax.set_xlabel('iterations')
ax.set_ylabel('f(x)')
ax.set_yscale('log')

opt_names = ['Adam','Adam2']

for index, res in enumerate(results):
    ax = plot_results(res, ax, f"optimizer={opt_names[index]}")

ax.legend(loc='lower right', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)